In [ ]:
import ecole

In [ ]:
default_scip_params = {'separating/maxrounds': 0,
                         'separating/maxroundsroot': 0,
                         'separating/maxcuts': 0,
                         'separating/maxcutsroot': 0,
                         'presolving/maxrounds': 0,
                         'presolving/maxrestarts': 0,
                         'propagating/maxrounds':0,
                         'propagating/maxroundsroot':0,
                         'lp/initalgorithm':'d',
                         'lp/resolvealgorithm':'d',
                         'limits/time': 3600}

class EcoleBranching(ecole.environment.Branching):
    def __init__(
        self,
        observation_function='default',
        information_function='default',
        reward_function='default',
        scip_params='default',
        pseudo_candidates=False,
    ):
        self.pseudo_candidates = pseudo_candidates

        # init functions from strings if needed
        if reward_function == 'default':
            reward_function = ({
                     'num_nodes': -ecole.reward.NNodes(),
                     'lp_iterations': -ecole.reward.LpIterations(),
                     'solving_time': -ecole.reward.SolvingTime(),
                 })
        if information_function == 'default':
            information_function=({
                     'num_nodes': ecole.reward.NNodes().cumsum(),
                     'lp_iterations': ecole.reward.LpIterations().cumsum(),
                     'solving_time': ecole.reward.SolvingTime().cumsum(),
                 })
        if observation_function == 'default':    
            observation_function = (ecole.observation.NodeBipartite())
        if scip_params == 'default':
            scip_params = default_scip_params
        
        super(EcoleBranching, self).__init__(
            observation_function=observation_function,
            information_function=information_function,
            reward_function=reward_function,
            scip_params=scip_params,
            pseudo_candidates=pseudo_candidates,
        )
        
class EcoleConfiguring(ecole.environment.Configuring):
    def __init__(self,
                 observation_function='default',
                 information_function='default',
                 scip_params='default'):

        # init functions from strings if needed
        if observation_function == 'default':    
            observation_function=None
        if information_function == 'default':
            information_function=({
                     'num_nodes': ecole.reward.NNodes().cumsum(),
                     'lp_iterations': ecole.reward.LpIterations().cumsum(),
                     'solving_time': ecole.reward.SolvingTime().cumsum(),
                 })
        if scip_params == 'default':
            scip_params = default_scip_params

        super(EcoleConfiguring, self).__init__(observation_function=observation_function,
                                                                       information_function=information_function,
                                                                       scip_params=scip_params)

In [ ]:
num_instances = 10
seed = 0

# Default SCIP Branching

In [ ]:
ecole.seed(seed)

env = EcoleConfiguring(observation_function='default',
                                  information_function='default',
                                  scip_params='default')

instances = ecole.instance.SetCoverGenerator(n_rows=500, n_cols=1000, density=0.05)

In [ ]:
for i in range(num_instances):
    done = True
    while done:
        # find instance which is not pre-solved
        instance = next(instances)
        env.seed(seed)
        obs, action_set, reward, done, info = env.reset(instance)
    
    obs, action_set, reward, done, info = env.step({})
    print(f'Instance {i+1} num nodes with default SCIP: {info["num_nodes"]}')

# Strong Branching

In [ ]:
class StrongBranchingAgent:
    def __init__(self, pseudo_candidates=False, name='sb'):
        self.name = name
        self.pseudo_candidates = pseudo_candidates
        self.strong_branching_function = ecole.observation.StrongBranchingScores(pseudo_candidates=pseudo_candidates)

    def before_reset(self, model):
        """
        This function will be called at initialization of the environment (before dynamics are reset).
        """
        self.strong_branching_function.before_reset(model)
    
    def extract(self, model, done):
        return self.strong_branching_function.extract(model, done)

    def action_select(self, action_set, model, done, **kwargs):
        scores = self.extract(model, done)
        action_idx = scores[action_set].argmax()
        return action_set[action_idx], action_idx
    
agent = StrongBranchingAgent()

ecole.seed(seed)

env = EcoleBranching(observation_function='default',
                      information_function='default',
                      reward_function='default',
                      scip_params='default')

instances = ecole.instance.SetCoverGenerator(n_rows=500, n_cols=1000, density=0.05)

In [ ]:
for i in range(num_instances):
    done = True
    while done:
        # find instance which is not pre-solved
        instance = next(instances)
        agent.before_reset(instance)
        env.seed(seed)
        obs, action_set, reward, done, info = env.reset(instance)
    
    while not done:
        action, action_idx = agent.action_select(action_set=action_set, model=env.model, done=done)
        obs, action_set, reward, done, info = env.step(action)
    print(f'Instance {i+1} num nodes with Strong Branching: {info["num_nodes"]}')